# Project: Customer Relationship Prediction
Univ.AI <br>
AI-1 Cohort-5

## Name of the people who worked on the project:


Link to presentation: *link here*

Link to slides: *link here*

Link to poster: *link here*

Any other information:*link here*

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (confusion_matrix, auc, f1_score,roc_auc_score)
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

In [56]:
df = pd.read_csv("./orange_small_train.data", sep = "\t")


In [57]:
churn = pd.read_table('orange_small_train_churn.labels', header = None).loc[:, 0]


In [58]:
churn = pd.Series(churn)
df = df.assign(churn = churn)

In [59]:
s = df.shape[1] 

In [61]:
missing_perc = 20
min_count = int(((100 - missing_perc) / 100) * df.shape[0] + 1)
min_count

40001

In [62]:
#drop all missing and only continous variable
prefered_uniques = 10
# create a list of numeric values and less than prefered uniques categorical columns
fe_unique_columns = list(df.select_dtypes(include='number').columns) + list(
    (df.select_dtypes(include='object').nunique() < prefered_uniques).index[
        df.select_dtypes(include='object').nunique() < prefered_uniques])
# collect only numeric features
df_all_missing_drop_continous = df.dropna( axis=1,thresh=min_count).dropna(axis=0).select_dtypes(include=['number'])#.agg(['nunique','count'])
numeric_col_after_drop = df_all_missing_drop_continous.columns
# create df with categorical variables with less than 10 uniques
df_all_missing_dropped_prefered_unique_cat = df[fe_unique_columns].dropna( axis=1,thresh=min_count).dropna(axis=0)
feature_prefered_columns = list(df_all_missing_dropped_prefered_unique_cat.columns)
# creating dummy variables for the data with  categorical variables
df_encoded = pd.get_dummies(df_all_missing_dropped_prefered_unique_cat,drop_first=True)
df_numeric = df_all_missing_drop_continous
del df_all_missing_drop_continous

In [63]:
df.dropna(axis=1, thresh=min_count).dropna(axis=0).select_dtypes(include='object').agg(['nunique']) < prefered_uniques

,Var192,Var193,Var195,Var196,Var197,Var198,Var199,Var202,Var203,Var204,...,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var226,Var227,Var228
nunique,False,False,False,True,False,False,False,False,True,False,...,False,True,False,False,True,False,True,False,True,False


In [64]:
df.dropna(axis=1, thresh=min_count).dropna(axis=0)['Var196'].unique()


array(['1K8T', 'z3mO', 'mKeq', 'JA1C'], dtype=object)

In [65]:
df = df.fillna(df.mean())

In [66]:
df

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,churn
0,11.487179,0.004029,425.298387,0.125396,238793.32885,1526.000000,7.000000,NaN,48.145299,392605.656355,...,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
1,11.487179,0.004029,425.298387,0.125396,238793.32885,525.000000,0.000000,NaN,48.145299,392605.656355,...,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN,1
2,11.487179,0.004029,425.298387,0.125396,238793.32885,5236.000000,7.000000,NaN,48.145299,392605.656355,...,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN,-1
3,11.487179,0.004029,425.298387,0.125396,238793.32885,1326.437116,0.000000,NaN,48.145299,392605.656355,...,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
4,11.487179,0.004029,425.298387,0.125396,238793.32885,1029.000000,7.000000,NaN,48.145299,392605.656355,...,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,11.487179,0.004029,425.298387,0.125396,238793.32885,357.000000,0.000000,NaN,48.145299,392605.656355,...,EROH7Cg,LM8l689qOp,NaN,NaN,7FJQ,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
49996,11.487179,0.004029,425.298387,0.125396,238793.32885,1078.000000,0.000000,NaN,48.145299,392605.656355,...,GfSQowC,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,am7c,NaN,-1
49997,11.487179,0.004029,425.298387,0.125396,238793.32885,2807.000000,7.000000,NaN,48.145299,392605.656355,...,dh6qI2t,LM8l689qOp,NaN,ELof,fKCe,RAYp,TCU50_Yjmm6GIBZ0lL_,NaN,NaN,-1
49998,11.487179,0.004029,425.298387,0.000000,238793.32885,1326.437116,6.809496,NaN,48.145299,392605.656355,...,2fF2Oqu,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [67]:
df = df.dropna(axis=1)

In [68]:
for i in range(s-39, s+ 1):
    value = 0
    value = 'Var{}'.format(i)
    try:
        df.drop(value, axis=1, inplace=True)
    except:
        pass

c:\Users\munch\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [69]:
df_churn = df[df['churn']==1]
df_not = df[df['churn']==-1]
df_upsampled = resample(df_churn, replace=True, n_samples=df_not.shape[0])

df = pd.concat([df_not, df_upsampled])
df_X = df.drop(['churn'],axis=1)
df_y = df['churn']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(df_X,df_y ,test_size=0.2)


In [71]:
X_train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
2313,11.487179,0.004029,425.298387,0.125396,238793.32885,1946.0,0.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,162.000000,22007.045192
46094,11.487179,0.004029,425.298387,0.125396,238793.32885,875.0,7.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
30821,11.487179,0.004029,425.298387,0.125396,238793.32885,581.0,7.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
39518,11.487179,0.004029,425.298387,0.125396,238793.32885,322.0,0.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
5396,11.487179,0.004029,425.298387,0.125396,238793.32885,1064.0,7.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,180.000000,22007.045192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48423,11.487179,0.004029,425.298387,0.125396,238793.32885,294.0,0.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
14063,11.487179,0.004029,425.298387,0.125396,238793.32885,721.0,14.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
35721,11.487179,0.004029,425.298387,0.125396,238793.32885,70.0,7.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,246.000000,22007.045192
19954,11.487179,0.004029,425.298387,0.125396,238793.32885,2254.0,7.0,48.145299,392605.656355,8.625806,...,3.776755e+06,0.0,1.416638e+06,77773.795326,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192


In [81]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

c:\Users\munch\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LogisticRegression(solver='liblinear')

In [82]:
y_pred = model.predict(X_test)

In [83]:
accuracy_score(y_test, y_pred)

0.5725771638247356

In [84]:
roc_auc_score(y_test, y_pred)

0.5717244728393419